In [110]:
# Import modules
import numpy as np
import numpy.ma as ma
import netCDF4
import os
import csv
import dateutil.parser as parser
from datetime import datetime
import json
import pickle

import subprocess

In [2]:
def nearest_pnt(lat_f, lon_f, stn_lt, stn_ln):
    # Find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
    # Determines how close a point is to the particular latitude and longitude.
    abslat_f = np.abs(lat_f-stn_lt)
    abslon_f = np.abs(lon_f-stn_ln)
    
    # Now combine these two results.
    # use numpy.maximum, which takes two arrays and finds the local maximum.
    c_f = np.maximum(abslon_f, abslat_f)
    
    # Find index location of nearest grid location to the station by using the min function.
    # The latlon_idx is for a flattened array. Uncomment to use
    #latlon_idx = np.argmin(c)
    
    # Get the row/column index of the 2 D arrays like this
    x_f, y_f = np.where(c_f == np.min(c_f))
    return x_f[0], y_f[0]

def get_ncdata(nc_filename):
    # Open nc file
    nc = netCDF4.Dataset(os.path.join('./files' ,nc_filename), "r")
    # Point to groups
    nav = nc.groups['navigation_data']
    data = nc.groups['geophysical_data']
    # Pull nav variables
    lt = np.array(nav.variables['latitude'][:,:])
    ln = np.array(nav.variables['longitude'][:,:])
    nc.close()
    
    return lt, ln

In [212]:
def rrs_data(nc_filename, bnd):
    print('start rrs_data')
    nc = netCDF4.Dataset(os.path.join('./files' ,nc_filename), "r")
    data = nc.groups['geophysical_data']
    var_rrs = np.array(data.variables[bnd])
    # Rrs_412 = ma.masked_where(Rrs_412 < -32760.0, Rrs_412)
    list_rrs = []
    for i in range(0, len(x_list)):
        if x_list[i] > 0:
            v = var_rrs[x_list[i], y_list[i]]
            if v < -32760.0:
                list_rrs.append(-9999.)
            else:
                list_rrs.append(v)
        else:
            list_rrs.append(-9999.)
    
    nc.close()    
    print('end rrs_data')
    return list_rrs

In [200]:
def rrs_data_old(nc_filename, bnd):
    nc = netCDF4.Dataset(os.path.join('./files' ,nc_filename), "r")
    data = nc.groups['geophysical_data']
    var_rrs = np.array(data.variables[bnd])
    # Rrs_412 = ma.masked_where(Rrs_412 < -32760.0, Rrs_412)
    list_rrs = []
    for i in range(0, len(x_list)):
        if x_list[i] > 0:
            v = var_rrs[x_list[i], y_list[i]]
            print(v)
            list_rrs.append(v)
        else:
            list_rrs.append(-32760.0)
    
    nc.close()    
    print(list_rrs)
    return list_rrs

In [4]:
def get_csv_databyrow(sat_fls_csv):
    
    rws = []
    
    with open(sat_fls_csv, 'r') as satfiles: 
        # creating a csv reader object 
        csvreader = csv.reader(satfiles) 

        # extracting each data row one by one 
        for sat_rws in csvreader: 
            sat_rws.append(row)
    
    return rws

def get_csv_byrow(csvfl):
    
    rows = []
    with open(csvfl, 'r') as satinfo: 
        # creating a csv reader object 
        csvreader = csv.reader(satinfo) 

        # extracting field names through first row 
        fields = csvreader.next() 

        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(row) 

        return rows

In [181]:
def csv_prep(dict_lst):
    bands = ['412', '443', '469', '488', '531', '547', '555', '645', '667', '678']
    header = ['id', 'stn_lat', 'stn_lon', 'stn_time', 'poc',
              'lat_cntr', 'lon_cntr', 'nc_file']
    for b in bands:
        header.append(b+'_cntr')
        header.append(b+'mean')
        header.append(b+'stdev')
        header.append(b+'count')
    
    csv_lt = []
    csv_lt.append(header)
    for i in dict_lst:
        i_lst = []
        i_lst.append(i['id'])
        i_lst.append(i['stn_lat'])
        i_lst.append(i['stn_lon'])
        i_lst.append(i['stn_time'])
        i_lst.append(i['stn_poc'])

        i_lst.append(i['lats3x3_cntr'])
        i_lst.append(i['lons3x3_cntr'])
        i_lst.append(i['nc_file'])
        
        for b in bands:
            i_lst.append(i['Rrs_' + b + '_cntr'])
            i_lst.append(i['Rrs_' + b + 'mean'])
            i_lst.append(i['Rrs_' + b + 'stdev'])
            i_lst.append(i['Rrs_' + b + 'count'])

        csv_lt.append(i_lst)
    return csv_lt

In [213]:
dict_temp = {}
dict_list = []

file_suffix = '_modis_run1'

csv_save = 'results' + file_suffix + '.csv'
pkl_save = 'results' + file_suffix + '.pkl'

satfilename_csv = 'matchups_files' + file_suffix + '.csv'
sat_rws = get_csv_byrow(satfilename_csv)

matchup_csv = 'poc_matchups.csv'
matchup_rws = get_csv_byrow(matchup_csv)

for ln in matchup_rws:
    #ln = rows[3]
    stn_lat = float(ln[4])
    stn_lon = float(ln[5])
    dt = ln[6]
    tm = ln[7]
    poc = float(ln[8])
    unique_id = ln[9]

    stn_time = ' --stime ' + parser.parse(dt + ' ' + tm).isoformat()+'Z'
    stn_id = ' --id ' + unique_id
    
    print(' ')
     print('NEW FILE')
    matching_ids = [ln for ln in sat_rws if unique_id == ln[0]]
    
    for rw in matching_ids:
        dict_temp = {}
        sat_fl = rw[1]
        lats, lons = get_ncdata(sat_fl)
        x, y = nearest_pnt(lats, lons, stn_lat, stn_lon)
        
        y_list = []
        x_list = []
        lon_list = []
        lat_list = []
        
        count = 1
        for y_n in range(y+1, y-2, -1):
            #print (y_n)
            if y_n < lats.shape[1] and y_n > 0:
                lat_m = lats[:,y_n]
                lat_m = lat_m[np.newaxis, :]
                lon_m = lons[:,y_n]
                lon_m = lon_m[np.newaxis, :]
                idx_x, idx_y = nearest_pnt(lat_m, lon_m, stn_lat, stn_lon)
                #print('2', idx_x, idx_y, y_n)
                #print (lons[idx_y][y_n], lats[idx_y][y_n])
    
                for x_n in range (idx_y-1, idx_y+2, 1):
                    #print('_list', x_n, y_n)
                    if x_n < lats.shape[0] and x_n > 0: 
                        y_list.append(y_n)
                        x_list.append(x_n)
                        lon_list.append(lons[x_n][y_n])
                        lat_list.append(lats[x_n][y_n])
                        #print('latlon', lons[x_n][y_n])
                    else:
                        y_list.append(-9999.)
                        x_list.append(-9999.)
                        lon_list.append(-9999.)
                        lat_list.append(-9999.)                           
            else:
                y_list.extend((-9999., -9999., -9999.))
                x_list.extend((-9999., -9999., -9999.))
                lon_list.extend((-9999., -9999., -9999.))
                lat_list.extend((-9999., -9999., -9999.))  
                
        #print('x_list', x_list, 'y_list', y_list)
        print('id', unique_id + '_#' + str(count))
        print('file', sat_fl)
        print('sta_latlon', stn_lat, stn_lon)
        print('file_latlon', lat_list[4], lon_list[4])
        print('xy', x, y)
        print('x_list', x_list)
        print('y_list', y_list)
        print('lat_list', lat_list)
        print('lon_list', lon_list)
        print('lat_list', lat_list)
        print('lon_list', lon_list)

        dict_temp['x_idx'] = x_list
        dict_temp['y_idx'] = y_list
        dict_temp['lats3x3'] = lat_list
        dict_temp['lons3x3'] = lon_list
        dict_temp['lons3x3_cntr'] = lon_list[4]
        dict_temp['lats3x3_cntr'] = lat_list[4]
        dict_temp['id'] = unique_id + '_#' + str(count)
        dict_temp['stn_lat'] = stn_lat
        dict_temp['stn_lon'] = stn_lon
        dict_temp['stn_time'] = tm
        dict_temp['stn_poc'] = poc
        dict_temp['nc_file'] = sat_fl
        
        count += 1

        rrs_bands = ['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488',
                     'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645',
                     'Rrs_667', 'Rrs_678']

        for bn in rrs_bands:
            rrs_vals = rrs_data(sat_fl, bn)
            print(bn, rrs_vals)

            dict_temp[bn + '_list'] = rrs_vals
            dict_temp[bn + '_cntr'] = rrs_vals[4]

            rrs_vals_np = np.array(rrs_vals, dtype=float)
            rrs_vals_ma = ma.masked_where(rrs_vals_np < -9000., rrs_vals_np)
            dict_temp[bn + 'count'] = rrs_vals_ma.count()
            if rrs_vals_ma.count() > 0:
                dict_temp[bn + 'mean'] = rrs_vals_ma.mean()
                dict_temp[bn + 'stdev'] = rrs_vals_ma.std()
            else:
                dict_temp[bn + 'mean'] = -9999.
                dict_temp[bn + 'stdev'] = -9999.

    print(bn, rrs_vals)
    dict_list.append(dict_temp)

f = open(pkl_save,"wb")
pickle.dump(dict_list,f)
f.close()

csv_lst = csv_prep(dict_list)

myFile = open(csv_save, 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(csv_lst)


('ID', 'ANT23_1', '1')
('id', 'ANT23_1_#1')
('file', 'A2005288120000.L2_LAC_OC.nc')
('sta_latlon', 49.721, -3.738)
('file_latlon', 49.717644, -3.7441323)
('xy', 1276, 1337)
('x_list', [1274, 1275, 1276, 1275, 1276, 1277, 1281, 1282, 1283])
('y_list', [1338, 1338, 1338, 1337, 1337, 1337, 1336, 1336, 1336])
('lat_list', [49.669918, 49.68552, 49.701118, 49.702087, 49.717644, 49.733196, 49.736168, 49.751698, 49.767227])
('lon_list', [-3.7749984, -3.7866037, -3.7982316, -3.7325518, -3.7441323, -3.7557349, -3.6996164, -3.7111137, -3.7226331])
('lat_list', [49.669918, 49.68552, 49.701118, 49.702087, 49.717644, 49.733196, 49.736168, 49.751698, 49.767227])
('lon_list', [-3.7749984, -3.7866037, -3.7982316, -3.7325518, -3.7441323, -3.7557349, -3.6996164, -3.7111137, -3.7226331])
start rrs_data
end rrs_data
('Rrs_412', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_443', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -99

('id', 'ANT23_2_#1')
('file', 'A2005289142000.L2_LAC_OC.nc')
('sta_latlon', 48.817, -5.303)
('file_latlon', -9999.0, -9999.0)
('xy', 1427, 0)
('x_list', [1417, 1418, 1419, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('y_list', [1, 1, 1, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('lat_list', [47.732704, 47.750793, 47.768883, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('lon_list', [-6.3700604, -6.3701448, -6.3702111, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('lat_list', [47.732704, 47.750793, 47.768883, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('lon_list', [-6.3700604, -6.3701448, -6.3702111, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_412', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_443', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_469

('id', 'ANT23_5_#1')
('file', 'A2005293122000.L2_LAC_OC.nc')
('sta_latlon', 45.746, -5.525)
('file_latlon', 45.748859, -5.5172372)
('xy', 220, 1338)
('x_list', [213, 214, 215, 219, 220, 221, 216, 217, 218])
('y_list', [1339, 1339, 1339, 1338, 1338, 1338, 1337, 1337, 1337])
('lat_list', [45.699932, 45.715904, 45.731869, 45.810539, 45.748859, 45.764797, 45.77766, 45.793533, 45.809406])
('lon_list', [-5.5450726, -5.5547953, -5.564538, -5.552103, -5.5172372, -5.5268874, -5.4716744, -5.4813752, -5.4910955])
('lat_list', [45.699932, 45.715904, 45.731869, 45.810539, 45.748859, 45.764797, 45.77766, 45.793533, 45.809406])
('lon_list', [-5.5450726, -5.5547953, -5.564538, -5.552103, -5.5172372, -5.5268874, -5.4716744, -5.4813752, -5.4910955])
start rrs_data
end rrs_data
('Rrs_412', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_443', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_da

end rrs_data
('Rrs_412', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_443', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_469', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_488', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_531', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_547', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_555', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_645', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('R

('id', 'ANT23_10_#1')
('file', 'A2005303142500.L2_LAC_OC.nc')
('sta_latlon', 22.499, -20.501)
('file_latlon', 22.495798, -20.506344)
('xy', 1783, 933)
('x_list', [1781, 1782, 1783, 1782, 1783, 1784, 1782, 1783, 1784])
('y_list', [934, 934, 934, 933, 933, 933, 932, 932, 932])
('lat_list', [22.474802, 22.484392, 22.49398, 22.486216, 22.495798, 22.505381, 22.488035, 22.497612, 22.507191])
('lon_list', [-20.513945, -20.515762, -20.517582, -20.504526, -20.506344, -20.508163, -20.493305, -20.495121, -20.496939])
('lat_list', [22.474802, 22.484392, 22.49398, 22.486216, 22.495798, 22.505381, 22.488035, 22.497612, 22.507191])
('lon_list', [-20.513945, -20.515762, -20.517582, -20.504526, -20.506344, -20.508163, -20.493305, -20.495121, -20.496939])
start rrs_data
end rrs_data
('Rrs_412', [0.0026720017, 0.0024860017, 0.0023979992, 0.0025920011, 0.002319999, 0.0025059991, 0.0025880001, 0.0024180003, 0.0023359992])
start rrs_data
end rrs_data
('Rrs_443', [0.0031240024, 0.0030799992, 0.003010001, 0.0

...

end rrs_data
('Rrs_412', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_443', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_469', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_488', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_531', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_547', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_555', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_645', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('R

end rrs_data
('Rrs_443', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_469', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_488', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_531', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_547', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_555', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_645', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_667', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('R

end rrs_data
('Rrs_488', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_531', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_547', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_555', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_645', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_667', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_678', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('id', 'Mirai17_27_#1')
('file', 'A2017246141500.L2_LAC_OC.nc')
('sta_latlon', 71.75033333, -151.5016667)
('file_latlon', 71.755203, -151.49654)
('xy', 11

end rrs_data
('Rrs_547', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_555', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_645', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_667', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_678', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('id', 'Mirai17_32_#1')
('file', 'A2017247014500.L2_LAC_OC.nc')
('sta_latlon', 71.48883333, -152.3558333)
('file_latlon', -9999.0, -9999.0)
('xy', 0, 0)
('x_list', [-9999.0, -9999.0, 1, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('y_list', [-9999.0, -9999.0, 1, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('lat_list', [-9999.0, -9999.0, 84.896973, -9999.0, -9999.0, -9999.0, -

end rrs_data
('Rrs_555', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_645', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_667', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
start rrs_data
end rrs_data
('Rrs_678', [-9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0, -9999.0])
('id', 'Mirai17_36_#1')
('file', 'A2017247114000.L2_LAC_OC.nc')
('sta_latlon', 71.354, -151.008)
('file_latlon', 71.376549, -151.02962)
('xy', 1923, 102)
('x_list', [1929, 1930, 1931, 1922, 1923, 1924, 1915, 1916, 1917])
('y_list', [103, 103, 103, 102, 102, 102, 101, 101, 101])
('lat_list', [71.28997, 71.330063, 71.315918, 71.390724, 71.376549, 71.362381, 71.439415, 71.425217, 71.411011])
('lon_list', [-150.97211, -150.97363, -150.97652, -151.02673, -151.02962, -151.0325, -151.09026, -151.09314, -151.09602])
('lat_lis

In [207]:
rrs_vals_ma.mean()

masked

In [202]:
for bn in rrs_bands:
    rrs_vals = rrs_data(sat_fl, bn)
    print(bn, rrs_vals)

    dict_temp[bn + '_list'] = rrs_vals
    dict_temp[bn + '_cntr'] = rrs_vals[4]

    rrs_vals_np = np.array(rrs_vals, dtype=float)
    rrs_vals_ma = ma.masked_where(rrs_vals_np < -32700.0, rrs_vals_np)
    dict_temp[bn + 'count'] = rrs_vals_ma.count()
    if rrs_vals_ma.count() > 0:
        dict_temp[bn + 'mean'] = rrs_vals_ma.mean()
        dict_temp[bn + 'stdev'] = rrs_vals_ma.std()
    else:
        dict_temp[bn + 'mean'] = None
        dict_temp[bn + 'stdev'] = None

-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
[-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0]
('Rrs_412', [-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0])
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
[-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0]
('Rrs_443', [-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0])
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
[-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0]
('Rrs_469', [-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0])
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
[-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0]
('Rrs_488', [-32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0, -32767.0, -32767.0, -32760.0])
-327

In [201]:
bn

'Rrs_678'

In [54]:
lat_m = lats[:,y_n]
print(len(lat_m))
lat_m = lat_m[np.newaxis, :]
print(len(lat_m[0]))

2030
2030


In [75]:
print(sat_rws[0][0])

ANT23_1


In [84]:
y_list

[None, None, 328, None, None, 327, None, None, 326]

In [114]:
#def main():

import csv
import dateutil.parser as parser
from datetime import datetime
import subprocess

sat_rws = [] 
# load sat files
with open('matchups_files_modis_run1.csv', 'r') as satfiles: 
    # creating a csv reader object 
    csvreader = csv.reader(satfiles) 

    # extracting each data row one by one 
    for sat_rws in csvreader: 
        sat_rws.append(row)
        
for ln in sat_rws:
    #ln = rows[3]
    sat_unqid = ln[0]
    sat_files = ln[1]

fields = [] 
rows = [] 
# load matchup info
with open('matchups6.csv', 'r') as satinfo: 
    # creating a csv reader object 
    csvreader = csv.reader(satinfo) 

    # extracting field names through first row 
    fields = csvreader.next() 

    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

for ln in rows:
    #ln = rows[3]
    llat = ln[4]
    llon = ln[5]
    dt = ln[6]
    tm = ln[7]
    poc = ln[8]
    unique_id = ln[9]

    stn_time = ' --stime ' + parser.parse(dt + ' ' + tm).isoformat()+'Z'
    stn_id = ' --id ' + unique_id

    print(myCmd)
    #subprocess.call(myCmd, shell=True)
        
#if __name__ == "__main__": main()

NameError: name 'row' is not defined

In [57]:
print(x, y)
print(len(lons), len(lats), lats.shape)

(2029, 1076)
(2030, 2030, (2030, 1354))


In [3]:
# station info
# Replace with pulling info from file
stn_lat = 55.250
stn_lon = -89.533
nc_file = 'A2015217083500.L2_LAC_OC.nc'

dict_temp = {}

In [4]:
# pull the nc file lat/lon arrays
lats, lons = get_ncdata(nc_file)

In [5]:
# Find the nearest lon/lat grid point to the station lat/lon
x, y = nearest_pnt(lats, lons, stn_lat, stn_lon)

In [6]:
# generate y, x 3x3 grid around center point
# make a function pass dict, x, y, stn_lat, stn_lon
y_list = []
x_list = []
lon_list = []
lat_list = []
for y_n in range(y+1, y-2, -1):
    #print (y_n)
    lat_m = lat[[y+1]]
    lon_m = lon[[y+1]]
    idx_y, idx_x = nearest_pnt(lat_m, lon_m, stn_lat, stn_lon)
    #print(idx_x, idx_y)
    for x_n in range (x-1, x+2, 1):
        y_list.append(y_n)
        x_list.append(x_n)
        lon_list.append(lon[y_n][x_n])
        lat_list.append(lat[y_n][x_n])
        
dict_temp['x_idx'] = x_list
dict_temp['y_idx'] = y_list
dict_temp['lats'] = lat_list
dict_temp['lons'] = lon_list
dict_temp['lon_cnt'] = lon_list[4]
dict_temp['lat_cnt'] = lat_list[4]
print(y_list, y)
print(lon_list)

([912, 912, 912, 911, 911, 911, 910, 910, 910], 911)
([1064, 1065, 1066, 1064, 1065, 1066, 1064, 1065, 1066], 1065)
[-85.670097, -85.647919, -85.625694, -85.663162, -85.640976, -85.618744, -85.656227, -85.634026, -85.611778]
[55.959248, 55.954506, 55.949749, 55.969368, 55.964634, 55.959885, 55.979485, 55.974762, 55.970024]


In [10]:
dict_temp

In [13]:
rrs_bands = ['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488',
             'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645',
             'Rrs_667', 'Rrs_678']

for n in rrs_bands:
    dict_temp = rrs_data(nc_file, n, dict_temp)

[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]


In [9]:
dict_temp

{'Rrs_412_cntr': None,
 'Rrs_412_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_412count': 0,
 'Rrs_412mean': None,
 'Rrs_443_cntr': None,
 'Rrs_443_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_443count': 0,
 'Rrs_443mean': None,
 'Rrs_469_cntr': None,
 'Rrs_469_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_469count': 0,
 'Rrs_469mean': None,
 'Rrs_488_cntr': None,
 'Rrs_488_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_488count': 0,
 'Rrs_488mean': None,
 'Rrs_531_cntr': None,
 'Rrs_531_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_531count': 0,
 'Rrs_531mean': None,
 'Rrs_547_cntr': None,
 'Rrs_547_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_547count': 0,
 'Rrs_547mean': None,
 'Rrs_555_cntr': None,
 'Rrs_555_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_555count': 0,
 'Rrs_555mean': None,
 'Rrs_645_cntr': None,
 'Rrs_645_l

In [ ]:
lat_m.shape

In [ ]:
# Open nc file
nc = netCDF4.Dataset(nc_filename, "r")
# Point to groups
nav = nc.groups['navigation_data']
data = nc.groups['geophysical_data']
# Pull nav variables
lat = np.array(nav.variables['latitude'][:,:])
lon = np.array(nav.variables['longitude'][:,:])
nc.close()

In [ ]:
# Pull nav variables
lat = np.array(nav.variables['latitude'][:,:])
lon = np.array(nav.variables['longitude'][:,:])

x_count = nav.variables['cntl_pt_cols'][:]

In [ ]:
nc.close()

In [8]:
rrs_bands = ['Rrs_412', 'Rrs_443', 'Rrs_469', 'Rrs_488',
             'Rrs_531', 'Rrs_547', 'Rrs_555', 'Rrs_645',
             'Rrs_667', 'Rrs_678']

for n in rrs_bands:
    nc = netCDF4.Dataset(nc_file, "r")
    data = nc.groups['geophysical_data']
    var_rrs = np.array(data.variables[n])
    # Rrs_412 = ma.masked_where(Rrs_412 < -32760.0, Rrs_412)
    list_rrs = []
    for i in range(0, len(x_list)):
        v = var_rrs[x_list[i], y_list[i]]
        if v < -32760.0:
            list_rrs.append(None)
        else:
            list_rrs.append(v)
        
    print(list_rrs)
    dict_temp[n + '_list'] = list_rrs
    dict_temp[n + '_cntr'] = list_rrs[4]
    
    list_rrs_np = np.array(list_rrs)
    list_rrs_ma = ma.masked_where(list_rrs_np == None, list_rrs_np)
    dict_temp[n + 'count'] = list_rrs_ma.count()
    if list_rrs_ma.count() > 0:
        dict_temp[n + 'mean'] = list_rrs_ma.mean()
    else:
        dict_temp[n + 'mean'] = None
    
    nc.close()

[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None]


In [7]:
dict_temp['x_idx'] = x_list
dict_temp['y_idx'] = y_list

In [ ]:
print (lat.min(), lat.max())
print (lon.min(), lon.max())

In [ ]:
stn_lat = 55.250
stn_lon = -89.533

In [ ]:
# Find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
# This tells us how close a point is to the particular latitude and longitude.

abslat = np.abs(lat-stn_lat)
abslon= np.abs(lon-stn_lon)


In [52]:
lll = [None, None, None, None, None, None, None, None, None]
llll= [9.9, None, None, None, None, None, None, None, 8.8]
list_rrs_np = np.array(list_rrs)
list_rrs_ma = ma.masked_where(list_rrs_np == None, list_rrs_np)
print(list_rrs_ma.mean())
print(list_rrs_ma.count())
'''
cc = np.array([None, None, 8.9, 6.])
cc = ma.masked_where(cc == None, cc)
print(cc.mean(), cc.count())
'''

--
0


'\ncc = np.array([None, None, 8.9, 6.])\ncc = ma.masked_where(cc == None, cc)\nprint(cc.mean(), cc.count())\n'

In [ ]:
# Now combine these two results.
# use numpy.maximum, which takes two arrays and finds the local maximum.

c = np.maximum(abslon,abslat)


In [ ]:
# You can find the index location on the grid of nearest grid location to the station by using the min function.
latlon_idx = np.argmin(c)


In [ ]:
# The latlon_idx is for a flattened array
# you can  get the row/column index of the 2 D arrays like this
x, y = np.where(c == np.min(c))
print(x, y)
print (lon[x, y], lat[x[0], y[0]])


In [ ]:
# TEST
idx_y1, idx_x1 = nearest_pnt(lat, lon)
print (idx_y1, idx_x1)



In [ ]:
# create array of 3x3 matrix around point
y_list = []
x_list = []
for y_n in range(y+1, y-2, -1):
    for x_n in range (x-1, x+2, 1):
        y_list.append(y_n)
        x_list.append(x_n)
print(x_list)
print(y_list)

In [ ]:
# load the data Rrs fields
Rrs_412 = np.array(data.variables['Rrs_412'])
# Rrs_412 = ma.masked_where(Rrs_412 < -32760.0, Rrs_412)
list_412 = []
for i in range(0, len(x_list)):
    list_412.append(Rrs_412[x_list[i], y_list[i]])
print(list_412)
list_412 = ma.masked_where(list_412 > -32760.0, list_412)

print(list_412.mean(), list_412.count())

In [ ]:
nn = None
if nn == None:
    print 'yes'

In [48]:
cc = np.array([None, None, 8.9, 6.])
cc = ma.masked_where(cc == None, cc)
print(cc.mean(), cc.count())

(7.45, 2)


In [ ]:
# get center lat value
lat_center = lat[y,x]
# get line above center
lat_m = np.squeeze(lat[y-1])
lon_m = np.squeeze(lon[y-1])
# find lat value in line that that is closest to center lat
idx = (np.abs(lat_m-lat_center)).argmin()

print (idx)
print (y,x)
print (lat[y[0],x[0]], lat_m[idx])


In [ ]:

lat_m = lat[y+1]
lon_m = lon[y+1]

abslat_m = np.abs(lat_m-lat[y,x])
abslon_m = np.abs(lon_m-lon[y,x])

c_m = np.maximum(abslon_m, abslat_m)
x_m, y_m = np.where(c_m == np.min(c_m))
print(x_m, y_m)
print (x,y)

In [ ]:
def nearest_pnt(lat_f, lon_f):
    #abslat_f = np.abs(lat_f-lat[y,x])
    abslat_f = np.abs(lat_f-stn_lat)
    #abslon_f = np.abs(lon_f-lon[y,x])
    abslon_f = np.abs(lon_f-stn_lon)

    c_f = np.maximum(abslon_f, abslat_f)
    x_f, y_f = np.where(c_f == np.min(c_f))
    return x_f, y_f

y_list = []
x_list = []
for y_n in range(y+1, y-2, -1):
    #print (y_n)
    lat_m = lat[[y+1]]
    lon_m = lon[[y+1]]
    idx_y, idx_x = nearest_pnt(lat_m, lon_m)
    #print(idx_x, idx_y)
    for x_n in range (x-1, x+2, 1):
        y_list.append(y_n)
        x_list.append(x_n)
print(y_list, y)
print(x_list, x)

In [ ]:
y_list = []
x_list = []
for y_n in range(y+1, y-2, -1):
    lat_m = lat[y_n]
    for x_n in range (x-1, x+2, 1):
        y_list.append(y_n)
        x_list.append(x_n)
print(x_list)
print(y_list)

In [ ]:
idx = (np.abs(lat_m-lat[y,x])).argmin()

In [ ]:
np.squeeze(lat_m[x]).min()

In [ ]:
lat[y].min()

In [ ]:
y

In [ ]:
list(lat[list(y), list(x)])[0]

In [ ]:
lat[y+1].shape

In [ ]:
lat.shape

In [ ]:
lat[[y]].shape

In [59]:
dict_temp

{'Rrs_412_cnt': None,
 'Rrs_412_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_412count': 0,
 'Rrs_412mean': None,
 'Rrs_443_cnt': None,
 'Rrs_443_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_443count': 0,
 'Rrs_443mean': None,
 'Rrs_469_cnt': None,
 'Rrs_469_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_469count': 0,
 'Rrs_469mean': None,
 'Rrs_488_cnt': None,
 'Rrs_488_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_488count': 0,
 'Rrs_488mean': None,
 'Rrs_531_cnt': None,
 'Rrs_531_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_531count': 0,
 'Rrs_531mean': None,
 'Rrs_547_cnt': None,
 'Rrs_547_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_547count': 0,
 'Rrs_547mean': None,
 'Rrs_555_cnt': None,
 'Rrs_555_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_555count': 0,
 'Rrs_555mean': None,
 'Rrs_645_cnt': None,
 'Rrs_645_list': [N

In [18]:
import csv

with open('matchups1.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter='\n')
    lns = []
    for row in readCSV:
        lns.extend(row)

In [24]:
import csv
import dateutil.parser as parser
from datetime import datetime
import subprocess

fields = [] 
rows = [] 
with open('matchups3.csv', 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = csvreader.next() 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

for ln in rows
    #ln = rows[3]
    llat = ln[4]
    llon = ln[5]
    dt = ln[6]
    tm = ln[7]
    poc = ln[8]
    unique_id = ln[9]

    stn_time = ' --stime ' + parser.parse(dt + ' ' + tm).isoformat()+'Z'
    stn_id = ' --id ID123 ' + unique_id
    slon = ' --slon ' + str(float(llat)-0.05)
    elon = ' --elon ' + str(float(llat)+0.05)
    slat = ' --slat ' + str(float(llon)-0.05)
    elat = ' --elat ' + str(float(llon)+0.05)
    
    myCmd_pre = 'python fd_matchup.py --sat modisa --data_type oc --max_time_diff 3 '
    myCmd = myCmd_pre + slon + elon + slat + elat + stn_time + stn_id
    
    subprocess.call(xxx, shell=True)

In [62]:
rows

[['26',
  'BIOSOPE',
  '',
  'MAR1',
  '-8.394',
  '-141.235',
  '10/26/2004',
  '20:44',
  '-999',
  'BIOSOPE_26'],
 ['27',
  'BIOSOPE',
  '',
  'MAR2',
  '-8.365',
  '-141.268',
  '10/27/2004',
  '22:27',
  '59.65065789',
  'BIOSOPE_27'],
 ['28',
  'BIOSOPE',
  '',
  'MAR3',
  '-8.312',
  '-141.279',
  '10/28/2004',
  '21:19',
  '52.44342105',
  'BIOSOPE_28'],
 ['29',
  'BIOSOPE',
  '',
  'MAR4',
  '-8.319',
  '-141.239',
  '10/29/2004',
  '22:21',
  '42.79660714',
  'BIOSOPE_29'],
 ['30',
  'BIOSOPE',
  '',
  'NUK1',
  '-8.977',
  '-140.108',
  '10/30/2004',
  '14:55',
  '50.06943662',
  'BIOSOPE_30'],
 ['31',
  'BIOSOPE',
  '',
  'HNL1',
  '-9.013',
  '-136.847',
  '10/31/2004',
  '22:14',
  '40.77625',
  'BIOSOPE_31'],
 ['32',
  'BIOSOPE',
  '',
  'HNL2',
  '-9.034',
  '-136.945',
  '11/1/2004',
  '20:40',
  '88.1759375',
  'BIOSOPE_32'],
 ['33',
  'BIOSOPE',
  '',
  'STB1',
  '-11.739',
  '-134.099',
  '11/3/2004',
  '18:09',
  '39.79223214',
  'BIOSOPE_33'],
 ['34',
  'BIOSOPE',

In [8]:
fields

['RST ID',
 'Cruise',
 'SIOstation',
 'StationAlias',
 'Lat_[degN]',
 'Lon_[degE]',
 'Date_UTC',
 'Time_UTC',
 'POC_[mg/m^3]']

In [35]:
#for ln in rows
ln = rows[3]
llat = ln[4]
llon = ln[5]
dt = ln[6]
tm = ln[7]
poc = ln[8]
unique_id = ln[9]

stn_time = ' --stime ' + parser.parse(dt + ' ' + tm).isoformat()+'Z'
stn_id = ' --id ' + unique_id
slon = ' --slon ' + str(float(llat)-0.05)
elon = ' --elon ' + str(float(llat)+0.05)
slat = ' --slat ' + str(float(llon)-0.05)
elat = ' --elat ' + str(float(llon)+0.05)

myCmd_pre = 'python fd_matchup.py --sat modisa --data_type oc --max_time_diff 3 '
myCmd = myCmd_pre + slon + elon + slat + elat + stn_time + stn_id



In [38]:
myCmd_pre = 'python fd_matchup.py --sat modisa --data_type oc --max_time_diff 3 '

In [39]:
myCmd = myCmd_pre + slon + elon + slat + elat + stn_time + stn_id

In [40]:
myCmd

'python fd_matchup.py --sat modisa --data_type oc --max_time_diff 3  --slon 46.155 --elon 46.255 --slat -4.165 --elat -4.065 --stime 2005-10-19T12:13:00Z --id ID123 ANT23_4'

In [24]:
'python fd_matchup.py --sat modisa --data_type oc --slat 34. --slon 0  --stime 2015-07-15T14:30:00Z --max_time_diff 3  --id ID123'


'2005-10-19T12:13:00Z'

In [26]:
stn_time

'2005-10-19T12:13:00Z'

In [ ]:
xxxx = 'python fd_matchup.py --sat modisa --data_type oc --slat 34. --slon 0  --stime 2015-07-15T14:30:00Z --max_time_diff 3  --id ID123'

In [ ]:
fd_matchup.py --sat=modist --stime=2015-11-15T09:00:00Z --etime=2015-11-17T09:00:00Z --slat=23.0 --elat=25.0 --slon=170.0 --elon=175.0

In [1]:
myid = "ID_123"
myFile = "A20180521modis.nc"

In [15]:
row1 = myid + "," + myFile + "\n"
try:
    f = open('csvfile.csv','a')
    f.write(row1) #Give your csv text here.  
    ## Python will convert \n to os.linesep
    f.close()
except:
    f = open('csvfile.csv','w')
    f.write(row1) #Give your csv text here.
    ## Python will convert \n to os.linesep
    f.close()

In [7]:
os.system('pwd')

0

In [20]:
datetime.now().isoformat()+'Z'

'2019-01-18T10:49:01.616294Z'

In [50]:
rows[27][5]

'-141.279'

In [53]:
rows[1]

['2',
 'ANT23',
 '2',
 '002-2',
 '48.817',
 '-5.303',
 '10/16/2005',
 '11:58',
 '77.491',
 'ANT23_2']

In [60]:
rows = []
fields = []
with open('matchups4.csv', 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = csvreader.next() 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

In [61]:
rows[1]

['27',
 'BIOSOPE',
 '',
 'MAR2',
 '-8.365',
 '-141.268',
 '10/27/2004',
 '22:27',
 '59.65065789',
 'BIOSOPE_27']

In [155]:
dict_list[5]

{'Rrs_412_cntr': None,
 'Rrs_412_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_412count': 0,
 'Rrs_412mean': None,
 'Rrs_443_cntr': None,
 'Rrs_443_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_443count': 0,
 'Rrs_443mean': None,
 'Rrs_469_cntr': None,
 'Rrs_469_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_469count': 0,
 'Rrs_469mean': None,
 'Rrs_488_cntr': None,
 'Rrs_488_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_488count': 0,
 'Rrs_488mean': None,
 'Rrs_531_cntr': None,
 'Rrs_531_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_531count': 0,
 'Rrs_531mean': None,
 'Rrs_547_cntr': None,
 'Rrs_547_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_547count': 0,
 'Rrs_547mean': None,
 'Rrs_555_cntr': None,
 'Rrs_555_list': [None, None, None, None, None, None, None, None, None],
 'Rrs_555count': 0,
 'Rrs_555mean': None,
 'Rrs_645_cntr': None,
 'Rrs_645_l

In [168]:
header = ['id', 'stn_lat', 'stn_lon', 'stn_time', 'poc',
          'lat_cnt', 'lon_cnt', 'nc_file',
          '412_cnt', '412mean', '412count',
          '443_cnt', '443mean', '443count',
          '469_cnt', '469mean', '469count',
          '488_cnt', '488mean', '488count',
          '531_cnt', '531mean', '531count',
          '547_cnt', '547mean', '547count',
          '555_cnt', '555mean', '555count',
          '645_cnt', '645mean', '645count',
          '667_cnt', '667mean', '667count',
          '678_cnt', '678mean', '678count']
csv_lst = []
csv_lst.append(header)
for i in dict_list:
    i_lst = []
    i_lst.append(i['id'])
    i_lst.append(i['stn_lat'])
    i_lst.append(i['stn_lon'])
    i_lst.append(i['stn_time'])
    i_lst.append(i['stn_poc'])
    
    i_lst.append(i['lats9x9_cnt'])
    i_lst.append(i['lons9x9_cnt'])
    i_lst.append(i['nc_file'])
    
for b in bands:
    i_lst.append(i['Rrs_' + b + '_cntr'])
    i_lst.append(i['Rrs_' + b + 'mean'])
    i_lst.append(i['Rrs_' + b + 'stdev'])
    i_lst.append(i['Rrs_' + b + 'count'])
    
    i_lst.append(i['Rrs_412_cntr'])
    i_lst.append(i['Rrs_412mean'])
    i_lst.append(i['Rrs_412count'])
    
    i_lst.append(i['Rrs_443_cntr'])
    i_lst.append(i['Rrs_443mean'])
    i_lst.append(i['Rrs_443count'])
     
    i_lst.append(i['Rrs_469_cntr'])
    i_lst.append(i['Rrs_469mean'])
    i_lst.append(i['Rrs_469count'])

    
    i_lst.append(i['Rrs_488_cntr'])
    i_lst.append(i['Rrs_488mean'])
    i_lst.append(i['Rrs_488count'])
    
    
    i_lst.append(i['Rrs_531_cntr'])
    i_lst.append(i['Rrs_531mean'])
    i_lst.append(i['Rrs_531count'])

    
    i_lst.append(i['Rrs_547_cntr'])
    i_lst.append(i['Rrs_547mean'])
    i_lst.append(i['Rrs_547count'])
    
    
    i_lst.append(i['Rrs_555_cntr'])
    i_lst.append(i['Rrs_555mean'])
    i_lst.append(i['Rrs_555count'])
    
    
    i_lst.append(i['Rrs_645_cntr'])
    i_lst.append(i['Rrs_645mean'])
    i_lst.append(i['Rrs_645count'])
    
    
    i_lst.append(i['Rrs_667_cntr'])
    i_lst.append(i['Rrs_667mean'])
    i_lst.append(i['Rrs_667count'])
    
    
    i_lst.append(i['Rrs_678_cntr'])
    i_lst.append(i['Rrs_678mean'])
    i_lst.append(i['Rrs_678count'])
    
    csv_lst.append(i_lst)

f = open("modis_results_run1.pkl","wb")
pickle.dump(dict_list,f)
f.close()
    
myFile = open('modis_results_run1.csv', 'w')  
with myFile:  
   writer = csv.writer(myFile)
   writer.writerows(csv_lst)

In [177]:
csv_lst[10][7]

'A2005303142500.L2_LAC_OC.nc'

In [160]:
dict_list[9]

{'Rrs_412_cntr': 0.002319999,
 'Rrs_412_list': [0.0026720017,
  0.0024860017,
  0.0023979992,
  0.0025920011,
  0.002319999,
  0.0025059991,
  0.0025880001,
  0.0024180003,
  0.0023359992],
 'Rrs_412count': 9,
 'Rrs_412mean': 0.0024795558,
 'Rrs_443_cntr': 0.0029700026,
 'Rrs_443_list': [0.0031240024,
  0.0030799992,
  0.003010001,
  0.0030659996,
  0.0029700026,
  0.0030659996,
  0.003076002,
  0.0029760003,
  0.0029300004],
 'Rrs_443count': 9,
 'Rrs_443mean': 0.0030331118,
 'Rrs_469_cntr': 0.0035680011,
 'Rrs_469_list': [0.0037360005,
  0.0037459992,
  0.0036260001,
  0.0036399998,
  0.0035680011,
  0.0035540015,
  0.0037919991,
  0.0035960004,
  0.0036239997],
 'Rrs_469count': 9,
 'Rrs_469mean': 0.0036535556,
 'Rrs_488_cntr': 0.0031180009,
 'Rrs_488_list': [0.0032540001,
  0.0031620003,
  0.0030840002,
  0.0032059997,
  0.0031180009,
  0.0031419992,
  0.0032400005,
  0.0030799992,
  0.0030400008],
 'Rrs_488count': 9,
 'Rrs_488mean': 0.0031473334,
 'Rrs_531_cntr': 0.0010800026,
 'Rrs

In [158]:
import pickle

f = open("matchup_results.pkl","wb")
pickle.dump(dict_list,f)
f.close()

In [159]:
with open('matchup_results.json', 'w') as fp:
    json.dump({'data':dict_list}, fp)



TypeError: 49.69553 is not JSON serializable

In [156]:
header = ['id', 'stn_lat', 'stn_lon', 'stn_time', 'poc',
          'lat_cnt', 'lon_cnt', 'nc_file',
          '412_cnt', '412mean', '412count',
          '443_cnt', '443mean', '443count',
          '469_cnt', '469mean', '469count',
          '488_cnt', '488mean', '488count',
          '531_cnt', '531mean', '531count',
          '547_cnt', '547mean', '547count',
          '555_cnt', '555mean', '555count',
          '645_cnt', '645mean', '645count',
          '667_cnt', '667mean', '667count',
          '678_cnt', '678mean', '678count']


In [179]:
bands = ['412', '443', '469', '488', '531', '547', '555', '645', '667', '678']
header = ['id', 'stn_lat', 'stn_lon', 'stn_time', 'poc',
          'lat_cnt', 'lon_cnt', 'nc_file']
for b in bands:
    header.append(b+'_cnt')
    header.append(b+'mean')
    header.append(b+'count')


In [184]:
dict_list

[{'Rrs_412_cntr': None,
  'Rrs_412_list': [None, None, None, None, None, None, None, None, None],
  'Rrs_412count': 0,
  'Rrs_412mean': None,
  'Rrs_412stdev': None,
  'Rrs_443_cntr': None,
  'Rrs_443_list': [None, None, None, None, None, None, None, None, None],
  'Rrs_443count': 0,
  'Rrs_443mean': None,
  'Rrs_443stdev': None,
  'Rrs_469_cntr': None,
  'Rrs_469_list': [None, None, None, None, None, None, None, None, None],
  'Rrs_469count': 0,
  'Rrs_469mean': None,
  'Rrs_469stdev': None,
  'Rrs_488_cntr': None,
  'Rrs_488_list': [None, None, None, None, None, None, None, None, None],
  'Rrs_488count': 0,
  'Rrs_488mean': None,
  'Rrs_488stdev': None,
  'Rrs_531_cntr': None,
  'Rrs_531_list': [None, None, None, None, None, None, None, None, None],
  'Rrs_531count': 0,
  'Rrs_531mean': None,
  'Rrs_531stdev': None,
  'Rrs_547_cntr': None,
  'Rrs_547_list': [None, None, None, None, None, None, None, None, None],
  'Rrs_547count': 0,
  'Rrs_547mean': None,
  'Rrs_547stdev': None,
  'R

In [ ]:
dict_list[0]